## KNN Classifier for Steel surface defect recognition

#### If you use this code, please cite our work: [[paper](https://doi.org/10.1016/j.promfg.2020.05.146)][[code](https://github.com/adipandas/one-shot-steel-surfaces)]

### Aditya M. Deshpande, Ali A. Minai, Manish Kumar, One-Shot Recognition of Manufacturing Defects in Steel Surfaces, Procedia Manufacturing, Volume 48, 2020, Pages 1064-1071, ISSN 2351-9789, https://doi.org/10.1016/j.promfg.2020.05.146.

#### MIT License

Copyright (c) 2020 Aditya M. Deshpande

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

### Note: for this case, the number of neighbors in KNN were set to `1` and the dataset was also configured accordingly.

In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import os

### KNN classifier arguments

#### NOTE: You may need to change the path in the below dictionary as per your dataset 

In [39]:
args = {'training_dataset': './datasets_all/knn_data/training',
        'testing_dataset': './datasets_all/knn_data/testing',
        'neighbors': 1,
        'jobs': 1}

### Convert image to Vector
Function to *resize* the image to a fixed size, then flatten the image into a list of raw pixel intensities

In [40]:
def image_to_feature_vector(image, size=(32, 32)):
    return cv2.resize(image, size).flatten()

### Extract a 3D color histogram from the HSV color space using the supplied number of `bins` per channel

In [41]:
def extract_color_histogram(image, bins=(8, 8, 8)):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

### Image paths

In [42]:
imagePaths = list(paths.list_images(args["training_dataset"]))
imagePaths_test = list(paths.list_images(args["testing_dataset"]))

### Helper function to prepare data
* Extract each image and corresponding class label

In [43]:
def prepare_data(imagePaths):
    rawImages, features, labels = [], [], []
    
    for (i, imagePath) in enumerate(imagePaths):    
        image = cv2.imread(imagePath)
        label = imagePath.split(os.path.sep)[-1].split(".")[0].split("_")[0]
        pixels = image_to_feature_vector(image)
        hist = extract_color_histogram(image)
        
        rawImages.append(pixels)
        features.append(hist)
        labels.append(label)
    
    return rawImages, features, labels

### Load training and testing data

In [44]:
train_images, train_features, train_labels = prepare_data(imagePaths)
test_images, test_features, test_labels = prepare_data(imagePaths_test)

print("[INFO] Shape of the image sampled from dataset:", train_images[0].shape)

IndexError: list index out of range

### Memory consumed by the raw images matrix and features matrix

##### Training set

In [ ]:
trainRI = np.array(train_images)
trainRL = np.array(train_labels)
trainF = np.array(train_features)
print("[INFO] pixels matrix: {:.2f}MB".format(trainRI.nbytes / (1024 * 1000.0)))
print("[INFO] features matrix: {:.2f}MB".format(trainF.nbytes / (1024 * 1000.0)))

[INFO] pixels matrix: 0.02MB
[INFO] features matrix: 0.01MB


##### Testing set

In [ ]:
testRI = np.array(test_images)
testRL = np.array(test_labels)
testF = np.array(test_features)
print("[INFO] pixels matrix: {:.2f}MB".format(testRI.nbytes / (1024 * 1000.0)))
print("[INFO] features matrix: {:.2f}MB".format(testF.nbytes / (1024 * 1000.0)))

[INFO] pixels matrix: 5.40MB
[INFO] features matrix: 3.60MB


### Train and evaluate the k-NN classifer on the raw pixel intensities

In [ ]:
print("[INFO] evaluating raw pixel accuracy...")
model = KNeighborsClassifier(n_neighbors=args["neighbors"], n_jobs=args["jobs"])
model.fit(trainRI, trainRL)
acc = model.score(testRI, testRL)

print("[INFO] raw pixel accuracy: {:.2f}%".format(acc * 100))

[INFO] evaluating raw pixel accuracy...
[INFO] raw pixel accuracy: 28.22%
